# Experimento 2 : Análisis de optimizadores para la red soco optimizada.

En este experimento comprobaremos que optimizador tienen un mejor desempeño teniendo en cuenta nuestro conjunto de datos y las mejoras implementadas a la red SOCO.

## Librerías usadas.

In [1]:
import tensorflow as tf

gpus= tf.config.experimental.list_physical_devices('GPU')
print(gpus)
tf.config.experimental.set_memory_growth(gpus[0], True)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import math 
from glob import glob
from matplotlib import pyplot as plt
import os
from tqdm import tqdm
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.utils import resample

## Definición de rutas

In [3]:
#Rutas de los datos.
 
data_dir = os.path.dirname(os.path.realpath("../TFG/Datos/HAM10000_metadata.csv"))



csv_path = os.path.realpath(data_dir + "/HAM10000_metadata.csv")

#Variables globales

altura = 50
longitud = 50
clases = 7


print(data_dir)

print(csv_path)



/home/antgarnie/Escritorio/TFG/Datos
/home/antgarnie/Escritorio/TFG/Datos/HAM10000_metadata.csv


## Creación del marco de datos.

In [4]:
#Inicializando el dataFrame

dataFrame=pd.read_csv(csv_path)

#Mezclando carpetas.

all_image_path = glob(os.path.join(data_dir, '*', '*'))
imageid_path_dict = {os.path.splitext(os.path.basename(x))[0]: x for x in all_image_path}

# Inicializando diccionario de categorías

lesion_type_dict = {
    'nv': 'Melanocytic nevi',
    'mel': 'Melanoma',
    'bkl': 'Benign keratosis ',
    'bcc': 'Basal cell carcinoma',
    'akiec': 'Actinic keratoses',
    'vasc': 'Vascular lesions',
    'df': 'Dermatofibroma'
}

#Añadiendo columnas al dataFrame para que sea más legible.

dataFrame['path'] = dataFrame['image_id'].map(imageid_path_dict.get)
dataFrame['cell_type'] = dataFrame['dx'].map(lesion_type_dict.get) 
dataFrame['cell_type_idx'] = pd.Categorical(dataFrame['cell_type']).codes
dataFrame.head()


,lesion_id,image_id,dx,dx_type,age,sex,localization,path,cell_type,cell_type_idx
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,/home/antgarnie/Escritorio/TFG/Datos/HAM10000_...,Benign keratosis,2
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,/home/antgarnie/Escritorio/TFG/Datos/HAM10000_...,Benign keratosis,2
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,/home/antgarnie/Escritorio/TFG/Datos/HAM10000_...,Benign keratosis,2
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,/home/antgarnie/Escritorio/TFG/Datos/HAM10000_...,Benign keratosis,2
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,/home/antgarnie/Escritorio/TFG/Datos/HAM10000_...,Benign keratosis,2


## Preparación de la red



In [10]:
def full_build_cnn_soco(withBatchNormalization = False, withDropout=False):
    
    model = tf.keras.Sequential()
    
    model.add(tf.keras.layers.Conv2D(32, (3,3),(1,1),input_shape=(altura,longitud,3)))
    model.add(tf.keras.layers.PReLU())
    if(withBatchNormalization):
        model.add(tf.keras.layers.BatchNormalization())

    model.add(tf.keras.layers.Conv2D(32, (3,3),(1,1)))
    model.add(tf.keras.layers.PReLU())
    if(withBatchNormalization):
        model.add(tf.keras.layers.BatchNormalization())
    
    model.add(tf.keras.layers.MaxPooling2D((2,2)))

 
    model.add(tf.keras.layers.Conv2D(64, (3,3)))
    model.add(tf.keras.layers.PReLU())
    if(withBatchNormalization):
        model.add(tf.keras.layers.BatchNormalization())
        
    model.add(tf.keras.layers.MaxPooling2D((2,2)))
    
    model.add(tf.keras.layers.Flatten())
    
    model.add(tf.keras.layers.Dense(128))
    model.add(tf.keras.layers.PReLU())
    if(withDropout):
        model.add(tf.keras.layers.Dropout(0.5))

    model.add(tf.keras.layers.Dense(clases,activation='softmax'))
    model.summary()
    return model

## Se procede a crear un método que permita balancear la carga de imágenes

In [6]:
def balanced_dataset(df):
    df_balanced = pd.DataFrame()
    #df = pd.DataFrame()
    
    for cat in df['cell_type_idx'].unique():
        temp = resample(df[df['cell_type_idx'] == cat], 
                        replace=True,     # sample with replacement
                        n_samples=2500,   # to match majority class
                        random_state=123) # reproducible results

        # Combine majority class with upsampled minority class
        df_balanced = pd.concat([df_balanced, temp])
 
    df_balanced['cell_type'].value_counts()

    return df_balanced

def load_img_data(size, df, balanced=False):
    """
        ..
        first we should normalize the image from 0-255 to 0-1
    """
    
    img_h, img_w = size, size
    imgs = []
    
    if balanced:
        df = balanced_dataset(df)
    
    image_paths = list(df['path'])

    for i in tqdm(range(len(image_paths))):
        img = cv2.imread(image_paths[i])
        img = cv2.resize(img, (img_h, img_w))
        img = img.astype(np.float32) / 255.
        #img = np.asarray(Image.open(image_paths[i]).resize((size,size)))
        imgs.append(img)

    imgs = np.stack(imgs, axis=0)
    print(imgs.shape)

    #imgs = imgs.astype(np.float32) / 255.
    
    return imgs, df['cell_type_idx'].values

## Cargamos los datos y creamos los casos a experimentar.

In [8]:
def load_general_data():
    
    imgs, target = load_img_data(altura, dataFrame, balanced=True)
    
    x_train, x_transferLearning, y_train, y_transferLearning = train_test_split(imgs, target, test_size=0.60)
       
    source_data = [ x_transferLearning , y_transferLearning ]
    target_data = [ x_train , y_train ]
    
    x_train,x_test,y_train,y_test = train_test_split(target_data[0], target_data[1], test_size=0.70)
    
    train_data = [x_train,y_train]
    test_data = [x_test,y_test]
    
    return source_data,train_data,test_data


def get_data_for_ex(source_data,train_data,test_data):
    
    x_train = source_data[0]
    y_train = source_data[1]
    
    x_retrain = train_data[0]
    y_retrain = train_data[1]
    
    percent = math.floor(len(test_data[0])/100*30)
       
    x_validation = test_data[0][0:percent]
    y_validation = test_data[1][0:percent]
    
    
    x_test = test_data[0][percent:-1]
    y_test = test_data[1][percent:-1]
    
    return x_train,x_retrain,x_test,x_validation,y_train,y_retrain,y_test,y_validation


###############################################################################################################
# Definimos 7 experimentos cada uno con un optimizador distinto y definimos el número de iteraciones          #
###############################################################################################################

ITERATIONS_PER_EXP = 5
BATCH_SIZE = 16
EPOCHS = 15
LEARNING_RATE=0.0001

def set_hiper_to_exp_1(BATCH_SIZE,EPOCHS,LEARNING_RATE):
    opt = tf.keras.optimizers.RMSprop(learning_rate=LEARNING_RATE)  
    return BATCH_SIZE,EPOCHS,opt

def set_hiper_to_exp_2(BATCH_SIZE,EPOCHS,LEARNING_RATE):
    opt = tf.keras.optimizers.SGD(learning_rate=LEARNING_RATE)
    return BATCH_SIZE,EPOCHS,opt
    
def set_hiper_to_exp_3(BATCH_SIZE,EPOCHS,LEARNING_RATE):
    opt = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE) 
    return BATCH_SIZE,EPOCHS,opt

def set_hiper_to_exp_4(BATCH_SIZE,EPOCHS,LEARNING_RATE):
    opt = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE,amsgrad=True)
    return BATCH_SIZE,EPOCHS,opt
    
def set_hiper_to_exp_5(BATCH_SIZE,EPOCHS,LEARNING_RATE):
    opt = tf.keras.optimizers.Adamax(learning_rate=LEARNING_RATE)
    return BATCH_SIZE,EPOCHS,opt

def set_hiper_to_exp_6(BATCH_SIZE,EPOCHS,LEARNING_RATE):
    opt = tf.keras.optimizers.Nadam(learning_rate=LEARNING_RATE)
    return BATCH_SIZE,EPOCHS,opt

def set_hiper_to_exp_7(BATCH_SIZE,EPOCHS,LEARNING_RATE):
    opt = tf.keras.optimizers.Adadelta(learning_rate=LEARNING_RATE)  
    return BATCH_SIZE,EPOCHS,opt

In [9]:
source_data,train_data,test_data = load_general_data()
x_train,x_retrain,x_test,x_validation,y_train,y_retrain,y_test,y_validation = get_data_for_ex(source_data,train_data,test_data)

100%|██████████| 17500/17500 [01:50<00:00, 157.75it/s]


(17500, 50, 50, 3)


In [ ]:

BATCH_SIZE,EPOCHS,opt = set_hiper_to_exp_1(BATCH_SIZE,EPOCHS,LEARNING_RATE)
res11,res12,evaluations,evaluations1 = run_experiment(2,EPOCHS)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 48, 48, 32)        896       
_________________________________________________________________
p_re_lu_4 (PReLU)            (None, 48, 48, 32)        73728     
_________________________________________________________________
batch_normalization_3 (Batch (None, 48, 48, 32)        128       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 46, 46, 32)        9248      
_________________________________________________________________
p_re_lu_5 (PReLU)            (None, 46, 46, 32)        67712     
_________________________________________________________________
batch_normalization_4 (Batch (None, 46, 46, 32)        128       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 23, 23, 32)       

Epoch 1/15
657/657 [==============================] - 4s 7ms/step - loss: 0.9232 - accuracy: 0.6914 - val_loss: 0.6044 - val_accuracy: 0.8163
Epoch 2/15
657/657 [==============================] - 4s 6ms/step - loss: 0.3162 - accuracy: 0.8961 - val_loss: 0.3953 - val_accuracy: 0.8803
Epoch 3/15
657/657 [==============================] - 4s 6ms/step - loss: 0.1521 - accuracy: 0.9497 - val_loss: 0.2651 - val_accuracy: 0.9163
Epoch 4/15
657/657 [==============================] - 4s 7ms/step - loss: 0.0725 - accuracy: 0.9779 - val_loss: 0.2711 - val_accuracy: 0.9136
Epoch 5/15
657/657 [==============================] - 4s 6ms/step - loss: 0.0452 - accuracy: 0.9864 - val_loss: 0.3164 - val_accuracy: 0.9075
Epoch 6/15
657/657 [==============================] - 4s 7ms/step - loss: 0.0259 - accuracy: 0.9922 - val_loss: 0.3995 - val_accuracy: 0.8871
Epoch 7/15
657/657 [==============================] - 5s 7ms/step - loss: 0.0143 - accuracy: 0.9957 - val_loss: 0.3930 - val_accuracy: 0.9163
Epoch 

In [ ]:
#Ejecutamos los experimentos


BATCH_SIZE,EPOCHS,opt = set_hiper_to_exp_2(BATCH_SIZE,EPOCHS,LEARNING_RATE)
res21,res22,evaluations,evaluations2 = run_experiment(2,EPOCHS)

BATCH_SIZE,EPOCHS,opt = set_hiper_to_exp_3(BATCH_SIZE,EPOCHS,LEARNING_RATE)
res31,res32,evaluations,evaluations3 = run_experiment(2,EPOCHS)

BATCH_SIZE,EPOCHS,opt = set_hiper_to_exp_4(BATCH_SIZE,EPOCHS,LEARNING_RATE)
res41,res42,evaluations,evaluations4 = run_experiment(2,EPOCHS)

BATCH_SIZE,EPOCHS,opt = set_hiper_to_exp_5(BATCH_SIZE,EPOCHS,LEARNING_RATE)
res51,res52,evaluations,evaluations5 = run_experiment(2,EPOCHS)

BATCH_SIZE,EPOCHS,opt = set_hiper_to_exp_6(BATCH_SIZE,EPOCHS,LEARNING_RATE)
res61,res62,evaluations,evaluations6 = run_experiment(2,EPOCHS)

BATCH_SIZE,EPOCHS,opt = set_hiper_to_exp_7(BATCH_SIZE,EPOCHS,LEARNING_RATE)
res71,res72,evaluations,evaluations7 = run_experiment(2,EPOCHS)

In [11]:
def run_train(nn_base_arch,epochs,checkpoint,dense = False):
    nn = select_network(nn_base_arch)
    
    if dense == True :
        model = build_dense(nn)
    else:
        model = build(nn)
        
    cpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint, monitor="val_loss", mode="min", save_best_only=True, verbose=0)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    
    history = model.fit(x_train, y_train,epochs=EPOCHS,callbacks=[cpoint],batch_size = BATCH_SIZE,verbose=0)
      
    evaluation = model.evaluate(x_test, y_test)
        
    return history,evaluation

def run_train_w_model(model,epochs,dense = False):
        
    #cpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint, monitor="val_loss", mode="min", save_best_only=True, verbose=0)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    
    history = model.fit(x_train, y_train,validation_data=(x_validation, y_validation),
                        epochs=EPOCHS,batch_size = BATCH_SIZE)
    
    evaluation = model.evaluate(x_test, y_test)
        
    return history,evaluation,model

def re_train(model,epocas):
    #checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint, monitor="loss", mode="min", save_best_only=True, verbose=0)
    history = model.fit(x_train, y_train,validation_data=(x_validation, y_validation),epochs=EPOCHS,batch_size = BATCH_SIZE,verbose=2)
    evaluation = model.evaluate(x_test, y_test)
    return history,evaluation

In [12]:
def run_experiment(index_model,epochs,iterations = ITERATIONS_PER_EXP):
    result = []
    result_post_tf = []
    evaluations = []
    evaluations_post_tf = []
    
    for i in range(iterations):
        #checkpoint ="../TFG/Modelos/balanced_model_"+nn_base_arch+"_exp4_v_"+str(i)+"_EXP0.h5"
        
        if index_model == 0:
            model = full_build_cnn_soco(withBatchNormalization = False, withDropout=True)
        if index_model == 1:
            model = full_build_cnn_soco(withBatchNormalization = True, withDropout=True)
        if index_model == 2:
            model = full_build_cnn_soco(withBatchNormalization = True, withDropout=False)
        
        h,e,tf_model = run_train_w_model(model,epochs)
        result.append(h)
        evaluations.append(e)
        
        print("--------------------------------------------------------------------------")

        layers = tf_model.layers[0:-1]
        for layer in layers:
            layer.trainable = False
        
        h_retrain,e_retrain = re_train(tf_model,epochs)
        
        result_post_tf.append(h_retrain)
        evaluations_post_tf.append(e_retrain)
        
        
        
        print("########################################################")
        print("Iteración "+str(i+1) +" de "+ str(iterations))
        print("########################################################")
        
    return result,result_post_tf,evaluations,evaluations_post_tf

## Métodos de representación gráfica

## Representamos gráficamente los resultados obtenidos al experimentar con el tamaño del lote

In [ ]:
def compute_values(resx1,resx2,i,val):
    res=[]
    if val != "loss":
        res.append(0.0)
    for e in resx1[i].history[val]:
        res.append(e)
    for e in resx2[i].history[val]:
        res.append(e)
    return res




def plot_acc_all_experiments(res11,res12,res21,res22,res31,res32,res41,res42,res51,res52,res61,res62,res71,res72,epochs):
    plt.figure(figsize=(16,10))
    
    acc = "accuracy"
    
    for i in range(5):
        
        precisiones = compute_values(res11,res12,i,acc)
            
        precisiones1 = compute_values(res21,res22,i,acc)
        
        precisiones2 = compute_values(res31,res32,i,acc)
        
        precisiones3 = compute_values(res41,res42,i,acc)
        
        precisiones4 = compute_values(res51,res52,i,acc)
        
        precisiones5 = compute_values(res61,res62,i,acc)
        
        precisiones6 = compute_values(res71,res72,i,acc)
        
        loss = compute_values(res11,res12,i,"loss")
        
        loss1 = compute_values(res21,res22,i,"loss")
        
        loss2 = compute_values(res31,res32,i,"loss")
        
        loss3 = compute_values(res41,res42,i,"loss")
        
        loss4 = compute_values(res51,res52,i,"loss")
        
        loss5 = compute_values(res61,res62,i,"loss")
        
        loss6 = compute_values(res71,res72,i,"loss")

    
        if i == 0:
            plt.plot(np.arange(0, epochs+1), precisiones[0:epochs+1], label="RMSProp",color='green')
            #plt.plot(np.arange(0, epochs+1), precisiones1[0:epochs+1], label="SGD",color='yellow')
            plt.plot(np.arange(0, epochs+1), precisiones2[0:epochs+1], label="Adam",color='red')
            plt.plot(np.arange(0, epochs+1), precisiones3[0:epochs+1], label="Amsgrad",color='aqua')
            #plt.plot(np.arange(0, epochs+1), precisiones4[0:epochs+1], label="Adamax",color='red')
            #plt.plot(np.arange(0, epochs+1), precisiones5[0:epochs+1], label="Nadam",color='deeppink')
            #plt.plot(np.arange(0, epochs+1), precisiones6[0:epochs+1], label="Adadelta",color='yellowgreen')
            
            plt.plot(np.arange(0, epochs+1), loss[0:epochs+1],color='green')
            #plt.plot(np.arange(0, epochs+1), loss1[0:epochs+1], label="SGD",color='yellow')
            plt.plot(np.arange(0, epochs+1), loss2[0:epochs+1],color='red')
            plt.plot(np.arange(0, epochs+1), loss3[0:epochs+1],color='aqua')
            #plt.plot(np.arange(0, epochs+1), loss4[0:epochs+1], label="Adamax",color='red')
            #plt.plot(np.arange(0, epochs+1), loss5[0:epochs+1], label="Nadam",color='deeppink')
            #plt.plot(np.arange(0, epochs+1), loss6[0:epochs+1], label="Adadelta",color='yellowgreen')
            
        else:
            
            plt.plot(np.arange(0, epochs+1), precisiones[0:epochs+1],color='green')
            #plt.plot(np.arange(0, epochs+1), precisiones1[0:epochs+1], label="SGD",color='yellow')
            plt.plot(np.arange(0, epochs+1), precisiones2[0:epochs+1],color='red')
            plt.plot(np.arange(0, epochs+1), precisiones3[0:epochs+1],color='aqua')
            #plt.plot(np.arange(0, epochs+1), precisiones4[0:epochs+1], label="Adamax",color='red')
            #plt.plot(np.arange(0, epochs+1), precisiones5[0:epochs+1], label="Nadam",color='deeppink')
            #plt.plot(np.arange(0, epochs+1), precisiones6[0:epochs+1], label="Adadelta",color='yellowgreen')
            
            plt.plot(np.arange(0, epochs+1), loss[0:epochs+1],color='green')
            #plt.plot(np.arange(0, epochs+1), loss1[0:epochs+1],color='yellow')
            plt.plot(np.arange(0, epochs+1), loss2[0:epochs+1],color='red')
            plt.plot(np.arange(0, epochs+1), loss3[0:epochs+1],color='aqua')
            #plt.plot(np.arange(0, epochs+1), loss4[0:epochs+1],color='red')
            #plt.plot(np.arange(0, epochs+1), loss5[0:epochs+1],color='deeppink')
            #plt.plot(np.arange(0, epochs+1), loss6[0:epochs+1],color='yellowgreen')
            
        del precisiones
        del precisiones1
        del precisiones2
        del precisiones3
        del precisiones4
        del precisiones5
        del precisiones6

        
    plt.title("Training Accuracy and Test Results")
    plt.xlabel("Epoch #")
    plt.ylabel("Loss/Accuracy")
    plt.legend()
    plt.savefig("Exp_2_Mix_Results.jpg")
    plt.show()


In [ ]:
plot_acc_all_experiments(res11,res12,res21,res22,res31,res32,res41,res42,res51,res52,res61,res62,res71,res72,29)

## Creamos marcos de datos para analizar los resultados de evaluar los modelos 

In [ ]:
precision1 = []
precision2 = []
precision3 = []
precision4 = []
precision5 = []
precision6 = []
precision7 = []

for e in evaluations1:
    element = e[1]
    precision1.append(element)

for e in evaluations2:
    element = e[1]
    precision2.append(element)
    
for e in evaluations3:
    element = e[1]
    precision3.append(element)

for e in evaluations4:
    element = e[1]
    precision4.append(element)
    
for e in evaluations5:
    element = e[1]
    precision5.append(element)

for e in evaluations6:
    element = e[1]
    precision6.append(element)
    
for e in evaluations7:
    element = e[1]
    precision7.append(element)

  
    
d = {'RMSProp': precision1,'SGD': precision2,'Adam': precision3,
     'Amsgrad': precision4,'Adamax': precision5,'Nadam': precision6,'Adadelta': precision7}
df = pd.DataFrame(data=d)

print(df.mean())